In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="google-t5/t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1076: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [12]:
import transformers
transformers.__version__

'4.44.2'

In [10]:
from huggingface_hub import notebook_login
notebook_login ()

In [ ]:
from huggingface_hub import list_datasets
datasets_list = list_datasets ()
num = 0
for dataset in datasets_list:
  print(dataset)
  if (num>10):
    break
    num +=1

Output hidden; open in https://colab.research.google.com to view.

In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

In [16]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

In [1]:
!pip install datasets


In [2]:
from datasets import load_dataset
haris = load_dataset("SkunkworksAI/reasoning-0.01")
haris

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/847 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/56.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29857 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 29857
    })
})

In [4]:
haris = load_dataset("SkunkworksAI/reasoning-0.01", split="train[:3%]")
haris

Dataset({
    features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
    num_rows: 896
})

In [5]:
haris.shape

(896, 4)

In [6]:
print(haris.description)

In [7]:
haris.features

{'instruction': Value(dtype='string', id=None),
 'reasoning': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'reasoning_chains': [{'step': Value(dtype='int64', id=None),
   'thought': Value(dtype='string', id=None)}]}

In [8]:
haris[0]

{'instruction': 'If a die is rolled three times, what is the probability of getting a sum of 11? None',
 'reasoning': '1. Understand the problem: We need to find the probability of getting a sum of 11 when rolling a die three times.\n2. Calculate total possible outcomes: A die has 6 faces, so for each roll, there are 6 possibilities. For three rolls, the total possible outcomes are 6^3 = 216.\n3. Identify favorable outcomes: List all combinations of rolls that result in a sum of 11. There are 18 such combinations.\n4. Calculate probability: Divide the number of favorable outcomes by the total possible outcomes: 18 / 216 = 1/12.\n5. Conclusion: The probability of getting a sum of 11 when rolling a die three times is 1/12.',
 'output': "To solve this problem, we need to find the number of favorable outcomes (getting a sum of 11) and divide it by the total possible outcomes when rolling a die three times.\n\nFirst, let's find the total possible outcomes. Since a die has six faces, there a

In [9]:
haris = haris.train_test_split(test_size = 0.2 ,seed = 42)
haris

DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 716
    })
    test: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 180
    })
})

In [10]:
def clean_txt(example):
    for txt in ["article", "highlights"]:
        example[txt] = example[txt].lower()  # Convert text to lowercase
        example[txt] = example[txt].replace("\\", "")  # Remove backslashes
        example[txt] = example[txt].replace("\'", "")  # Remove single quotes
        example[txt] = example[txt].replace("\n", "")  # Remove newlines
        example[txt] = example[txt].replace("\r", "")  # Remove carriage returns
        example[txt] = example[txt].replace("\"", "")  # Remove double quotes
        example[txt] = example[txt].replace("-", "")  # Remove hyphens

    return example


In [14]:
print(haris.column_names)


{'train': ['instruction', 'reasoning', 'output', 'reasoning_chains'], 'test': ['instruction', 'reasoning', 'output', 'reasoning_chains']}


In [16]:
print(haris.column_names)


{'train': ['instruction', 'reasoning', 'output', 'reasoning_chains'], 'test': ['instruction', 'reasoning', 'output', 'reasoning_chains']}


In [17]:
def clean_txt(example):
    for txt in ["text", "highlights"]:  # Update with actual column names
        if txt in example:
            example[txt] = example[txt].lower()  # Convert text to lowercase
            example[txt] = example[txt].replace("\\", "")  # Remove backslashes
            example[txt] = example[txt].replace("\'", "")  # Remove single quotes
            example[txt] = example[txt].replace("\n", "")  # Remove newlines
            example[txt] = example[txt].replace("\r", "")  # Remove carriage returns
            example[txt] = example[txt].replace("\"", "")  # Remove double quotes
            example[txt] = example[txt].replace("-", "")  # Remove hyphens

    return example


In [18]:
cleaned_haris = haris.map(clean_txt)
cleaned_haris


Map:   0%|          | 0/716 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 716
    })
    test: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 180
    })
})

In [22]:
# For example, accessing the 7th entry in the 'instruction' column
haris['train']['instruction'][7]


"Mr. Nance: Ms. Chan said that she retired from Quad Cities Corporation, and had received a watch and a wonderful party as thanks for her 40 years of loyal service. But I overheard a colleague of hers say that Ms. Chan will be gone for much of the next year on business trips and is now working harder than she ever did before; that does not sound like retirement to me. At least one of them is not telling the truth. Mr. Nance's reasoning is flawed because it\nA: is based in part on hearsay\nB: draws a conclusion based on equivocal language\nC: criticizes Ms. Chan rather than the claims she made\nD: fails to infer that Ms. Chan must be a person of superior character, given her long loyal service Choose A, B, C or D as your solution."

In [23]:
# For example, accessing the 7th entry in the 'instruction' column
cleaned_haris['train']['instruction'][7]


"Mr. Nance: Ms. Chan said that she retired from Quad Cities Corporation, and had received a watch and a wonderful party as thanks for her 40 years of loyal service. But I overheard a colleague of hers say that Ms. Chan will be gone for much of the next year on business trips and is now working harder than she ever did before; that does not sound like retirement to me. At least one of them is not telling the truth. Mr. Nance's reasoning is flawed because it\nA: is based in part on hearsay\nB: draws a conclusion based on equivocal language\nC: criticizes Ms. Chan rather than the claims she made\nD: fails to infer that Ms. Chan must be a person of superior character, given her long loyal service Choose A, B, C or D as your solution."

In [24]:
haris['train']['reasoning'][7]


"1: Ms. Chan claims she retired from Quad Cities Corporation after 40 years of service.\n2: Mr. Nance overheard a colleague say Ms. Chan will be on business trips and working harder than before.\n3: Mr. Nance believes at least one of them is not telling the truth.\n4: The reasoning flaw options are A: hearsay, B: equivocal language, C: criticizing Ms. Chan, or D: failing to infer superior character.\n5: Mr. Nance's reasoning is not based on hearsay, as he directly overheard the colleague's statement.\n6: Mr. Nance's reasoning does not criticize Ms. Chan, but questions the truth of the statements.\n7: Mr. Nance's reasoning does not involve inferring Ms. Chan's character.\n8: The remaining option is B: equivocal language, which means Mr. Nance's conclusion is based on unclear or ambiguous language.\n9: The colleague's statement about business trips and working harder could be interpreted in different ways, making it equivocal.\n10: Therefore, the correct answer is B: Mr. Nance's reasonin

In [25]:
cleaned_haris['train']['reasoning'][7]


"1: Ms. Chan claims she retired from Quad Cities Corporation after 40 years of service.\n2: Mr. Nance overheard a colleague say Ms. Chan will be on business trips and working harder than before.\n3: Mr. Nance believes at least one of them is not telling the truth.\n4: The reasoning flaw options are A: hearsay, B: equivocal language, C: criticizing Ms. Chan, or D: failing to infer superior character.\n5: Mr. Nance's reasoning is not based on hearsay, as he directly overheard the colleague's statement.\n6: Mr. Nance's reasoning does not criticize Ms. Chan, but questions the truth of the statements.\n7: Mr. Nance's reasoning does not involve inferring Ms. Chan's character.\n8: The remaining option is B: equivocal language, which means Mr. Nance's conclusion is based on unclear or ambiguous language.\n9: The colleague's statement about business trips and working harder could be interpreted in different ways, making it equivocal.\n10: Therefore, the correct answer is B: Mr. Nance's reasonin

In [26]:
EXAMPLE_TEXT_INDEX = 33
example_text = cleaned_haris["test"]["instruction"][EXAMPLE_TEXT_INDEX]
example_text

"The university' s purchasing department is highly efficient overall. We must conclude that each of its twelve staff members is highly efficient. Which one of the following arguments exhibits flawed reasoning most similar to that exhibited by the argument above?\nA: The outside audit of our public relations department has exposed serious deficiencies in the competence of each member of that department. We must conclude that the department is inadequate for our needs.\nB: The employees at this fast-food restaurant are the youngest and most inexperienced of any fast-food workers in the city. Given this, it seems obvious that customers will have to wait longer for their food at this restaurant than at others.\nC: This supercomputer is the most sophisticated -- and the most expensive -- ever built. It must be that each of its components is the most sophisticated and expensive available.\nD: Literature critics have lavished praise on every chapter of this book. In light of their reviews, on

In [28]:
MODEL_NAME = "google-t5/t5-small"

In [31]:
from transformers import pipeline
prefix = "summarize: "
summarizer = pipeline("summarization", model = MODEL_NAME )
summary_txt =summarizer(prefix + example_text)
summary_txt


[{'summary_text': "the university' s purchasing department is highly efficient overall . the audit of the public relations department has exposed serious deficiencies in the competence of each member of that department ."}]

In [33]:
summarizer = pipeline("summarization", model = MODEL_NAME , truncation = True )
summary_txt =summarizer(prefix + example_text)
summary_txt


[{'summary_text': "the university' s purchasing department is highly efficient overall . the audit of the public relations department has exposed serious deficiencies in the competence of each member of that department ."}]

In [35]:
print(cleaned_haris["test"].column_names)


['instruction', 'reasoning', 'output', 'reasoning_chains']


In [37]:
ref_txt = cleaned_haris["test"]["instruction"][EXAMPLE_TEXT_INDEX]
ref_txt

"The university' s purchasing department is highly efficient overall. We must conclude that each of its twelve staff members is highly efficient. Which one of the following arguments exhibits flawed reasoning most similar to that exhibited by the argument above?\nA: The outside audit of our public relations department has exposed serious deficiencies in the competence of each member of that department. We must conclude that the department is inadequate for our needs.\nB: The employees at this fast-food restaurant are the youngest and most inexperienced of any fast-food workers in the city. Given this, it seems obvious that customers will have to wait longer for their food at this restaurant than at others.\nC: This supercomputer is the most sophisticated -- and the most expensive -- ever built. It must be that each of its components is the most sophisticated and expensive available.\nD: Literature critics have lavished praise on every chapter of this book. In light of their reviews, on

In [39]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [41]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=85eecef97c43a20e39afcff7db38755809806a4635e34ac32a2d4841e01eb439
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [42]:
import evaluate
rouge = evaluate.load("rouge")
rouge


EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [43]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"


In [45]:
results = rouge.compute(predictions=[ex_1], references=[reference])
results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

In [46]:
results = rouge.compute(predictions=[ex_2], references=[reference])
results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

In [48]:
article_texts = cleaned_haris["test"]["instruction"]
article_summaries = cleaned_haris["test"]["output"]

In [49]:
from tqdm import tqdm

candidate_summaries = []
prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):
    candidate = summarizer(prefix + text)
    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [02:27<00:00,  2.96s/it]


In [50]:
result_agg = rouge.compute(predictions=candidate_summaries,
                           references=article_summaries[:50],
                           use_stemmer=True)

result_agg

{'rouge1': 0.13812124829000874,
 'rouge2': 0.06311749760966806,
 'rougeL': 0.10595787442168483,
 'rougeLsum': 0.11851504122380038}

In [51]:
result_unagg = rouge.compute(predictions =candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                              use_aggregator =False)
result_unagg


{'rouge1': [0.08695652173913043,
  0.13138686131386862,
  0.08121827411167512,
  0.1726618705035971,
  0.4347826086956522,
  0.0,
  0.1168831168831169,
  0.0,
  0.0709812108559499,
  0.0,
  0.02708803611738149,
  0.15384615384615385,
  0.1282051282051282,
  0.0860215053763441,
  0.48192771084337344,
  0.17435897435897438,
  0.026936026936026935,
  0.2545454545454546,
  0.24691358024691357,
  0.2077922077922078,
  0.1308411214953271,
  0.0,
  0.0,
  0.11594202898550725,
  0.1846153846153846,
  0.35555555555555557,
  0.08695652173913043,
  0.052959501557632405,
  0.0,
  0.0,
  0.1152542372881356,
  0.05194805194805195,
  0.25274725274725274,
  0.0,
  0.18181818181818182,
  0.11578947368421053,
  0.29885057471264365,
  0.12162162162162163,
  0.0,
  0.0,
  0.2,
  0.30927835051546393,
  0.33043478260869563,
  0.29333333333333333,
  0.2285714285714286,
  0.0625,
  0.0,
  0.0,
  0.4768211920529802,
  0.09090909090909091],
 'rouge2': [0.035398230088495575,
  0.059259259259259255,
  0.0,
  0.01

In [52]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(48, 2)

In [53]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,the expressions are obtained by writing multip...,"Instead of computing $A$ and $B$ separately, w..."
1,implement a Golang function that takes a strin...,"package main\n\nimport (\n\t""fmt""\n)\n\n// lon..."
2,if Emily Dickinson wrote a poem about social m...,Within the glowing Screen I dwell —\nA solitar...
3,k steps rotate the array to the right . the in...,"```python\ndef rotate(nums, k):\n n = len(n..."
4,the probability that Asha will lose a game is ...,Since the sum of the probabilities of all poss...


In [54]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(38, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(38, "Reference_summaries"))

Actual Summary
Southern Airways has a far worse safety record than Air Dacentaria over the past few years . it is worth paying extra to have a safer flight from Pederton to dacenta .
Reference Summary
B


In [55]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(12, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(12, "Reference_summaries"))

Actual Summary
the numbers are equal to 1, 2, 3, 4, in some order . [ab + bc + cd + da.]
Reference Summary
I notice that this expression is symmetric in $a,$ $b,$ $c,$ $d,$ so the order of the numbers does not matter for the sum. I also notice that this expression is the same as $(a + c)(b + d),$ by expanding the product. So I want to maximize $(a + c)(b + d),$ given that $a + c + b + d = 1 + 2 + 3 + 4 = 10.$ So I want to find two pairs of numbers from 1, 2, 3, 4 that add up to the same value and are as close as possible to 5, which is half of 10. I see that one such pair is 2 and 3, which add up to 5. The other pair must be 1 and 4, which also add up to 5. Therefore, the largest possible value of $(a + c)(b + d)$ is $5 \times 5 = 25.$ 


In [56]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [57]:
tokenizer(["four-time defending champion"])

{'input_ids': [[662, 18, 715, 3, 20309, 6336, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [58]:
tokens = tokenizer.tokenize("four-time defending champion")

tokens

['▁four', '-', 'time', '▁', 'defending', '▁champion']

In [59]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

[662, 18, 715, 3, 20309, 6336]

In [60]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [61]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["highlights"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [63]:
print(cleaned_haris.column_names)


{'train': ['instruction', 'reasoning', 'output', 'reasoning_chains'], 'test': ['instruction', 'reasoning', 'output', 'reasoning_chains']}


In [65]:
def preprocess_function(examples):
    return tokenizer(examples['article'], truncation=True, padding='max_length')


In [66]:
def preprocess_function(examples):
    return tokenizer(examples['instruction'], truncation=True, padding='max_length')


In [67]:
tokenized_haris = cleaned_haris.map(preprocess_function, batched=True)
tokenized_haris


Map:   0%|          | 0/716 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains', 'input_ids', 'attention_mask'],
        num_rows: 716
    })
    test: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains', 'input_ids', 'attention_mask'],
        num_rows: 180
    })
})

In [68]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [69]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [70]:
from huggingface_hub import notebook_login

notebook_login()

In [73]:
{
    "input_ids": [...],  # Encoder input
    "decoder_input_ids": [...],  # Decoder input
    "attention_mask": [...],  # Attention mask
    "labels": [...]  # Labels for training
}


{'input_ids': [Ellipsis],
 'decoder_input_ids': [Ellipsis],
 'attention_mask': [Ellipsis],
 'labels': [Ellipsis]}

In [78]:
{
    "input_ids": [...],           # Input sequence for the encoder
    "attention_mask": [...],      # Attention mask for the encoder
    "decoder_input_ids": [...],   # Input sequence for the decoder
    "labels": [...]                # Ground truth labels for training
}


{'input_ids': [Ellipsis],
 'attention_mask': [Ellipsis],
 'decoder_input_ids': [Ellipsis],
 'labels': [Ellipsis]}

In [80]:
sources = [
    "Example input text 1",
    "Example input text 2",
    "Example input text 3",
    # Add more input texts as needed
]

targets = [
    "Example output text 1",
    "Example output text 2",
    "Example output text 3",
    # Add more target texts as needed
]


In [81]:
from datasets import Dataset

dataset = Dataset.from_dict({"source": sources, "target": targets})


In [82]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def encode_examples(examples):
    model_inputs = tokenizer(examples['source'], max_length=512, truncation=True)

    # Set up the decoder inputs (usually shifted labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], max_length=512, truncation=True)

    model_inputs["decoder_input_ids"] = labels["input_ids"]
    model_inputs["labels"] = labels["input_ids"]  # Use this or a different labels setting as needed

    return model_inputs

dataset = dataset.map(encode_examples, batched=True)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [84]:
# Assuming you have validation data
eval_sources = [
    "Example eval input text 1",
    "Example eval input text 2",
    # Add more eval input texts as needed
]

eval_targets = [
    "Example eval output text 1",
    "Example eval output text 2",
    # Add more eval target texts as needed
]

# Create the evaluation dataset
eval_dataset = Dataset.from_dict({"source": eval_sources, "target": eval_targets})
eval_dataset = eval_dataset.map(encode_examples, batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [85]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,  # Training dataset
    eval_dataset=eval_dataset,  # Evaluation dataset
)


In [86]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,7.740855
2,No log,7.599576
3,No log,7.539557


TrainOutput(global_step=3, training_loss=5.434546788533528, metrics={'train_runtime': 10.1632, 'train_samples_per_second': 0.886, 'train_steps_per_second': 0.295, 'total_flos': 11895275520.0, 'train_loss': 5.434546788533528, 'epoch': 3.0})

In [87]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

# Sample training data
sources = ["train input 1", "train input 2"]
targets = ["train output 1", "train output 2"]

# Sample evaluation data
eval_sources = ["eval input 1", "eval input 2"]
eval_targets = ["eval output 1", "eval output 2"]

# Create datasets
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def encode_examples(examples):
    model_inputs = tokenizer(examples['source'], max_length=512, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], max_length=512, truncation=True)

    model_inputs["decoder_input_ids"] = labels["input_ids"]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Training dataset
dataset = Dataset.from_dict({"source": sources, "target": targets})
dataset = dataset.map(encode_examples, batched=True)

# Evaluation dataset
eval_dataset = Dataset.from_dict({"source": eval_sources, "target": eval_targets})
eval_dataset = eval_dataset.map(encode_examples, batched=True)

# Model and training arguments
model = T5ForConditionalGeneration.from_pretrained("t5-small")
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,5.902650
2,No log,5.879074
3,No log,5.869569


TrainOutput(global_step=3, training_loss=6.464654286702474, metrics={'train_runtime': 10.8889, 'train_samples_per_second': 0.551, 'train_steps_per_second': 0.276, 'total_flos': 6344146944.0, 'train_loss': 6.464654286702474, 'epoch': 3.0})

In [88]:
trainer.push_to_hub()

events.out.tfevents.1727285836.dc150827f52e.24455.2:   0%|          | 0.00/5.71k [00:00<?, ?B/s]

events.out.tfevents.1727285955.dc150827f52e.24455.3:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1727286001.dc150827f52e.24455.4:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harismunir/results/commit/a4aea0a3ba451c4c06cdeec851519ae5b953fe30', commit_message='End of training', commit_description='', oid='a4aea0a3ba451c4c06cdeec851519ae5b953fe30', pr_url=None, pr_revision=None, pr_num=None)